<a href="https://colab.research.google.com/github/ymuto0302/PJ2024/blob/main/PJ_demographics(%E6%BC%94%E7%BF%92%E7%94%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[PJ学習]日本の人口動態データを用いた演習課題

### 準備
特にデータの前処理は（慣れないと）大変だから，コードを示しておく。

In [ ]:
# グラフでの日本語表示を可能とする
!pip install japanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=bcce40d3efe298de9897e860184a6b28accf2962cce6ec6b9c4784b414910544
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize-matplotlib


In [ ]:
# グラフでの日本語表示を可能とする
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
# 「全国，外国，不詳」など，余分なデータを削除するため，都道府県名の一覧を準備 → 後で使う
prefs = ['北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県',
         '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', '東京都', '神奈川県',
         '新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県', '岐阜県',
         '静岡県', '愛知県', '三重県', '滋賀県', '京都府', '大阪府', '兵庫県',
         '奈良県', '和歌山県', '鳥取県', '島根県', '岡山県', '広島県', '山口県',
         '徳島県', '香川県', '愛媛県', '高知県', '福岡県', '佐賀県', '長崎県',
         '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県']
len(prefs)

47

In [ ]:
import pandas as pd

# データ読み込み
df = pd.read_csv("FEH_00450011_240612201217.csv", skiprows=12)
df.head()

,時間軸(年次) コード,時間軸(年次),都道府県（特別区－指定都市再掲） コード,都道府県（特別区－指定都市再掲）,/人口動態総覧,人口【人】,出生数【人】,死亡数【人】,乳児死亡数【人】,新生児死亡数【人】,...,新生児死亡率【出生千対】,自然増減率【人口千対】,死産率【出産千対】,自然死産率【出産千対】,人工死産率【出産千対】,周産期死亡率【出産千対】,満22週以後の死産率【出産千対】,早期新生児死亡率【出生千対】,婚姻率【人口千対】,離婚率【人口千対】
0,2022000000,2022年,0,全国,NaN,"122,030,523","770,759","1,569,050","1,356",609,...,0.8,-6.5,19.3,9.4,9.9,3.3,2.7,0.6,4.1,1.47
1,2022000000,2022年,1000,北海道,NaN,"5,098,000","26,407","74,437",57,33,...,1.2,-9.4,21.7,8.7,13.0,3.1,2.2,0.8,3.7,1.65
2,2022000000,2022年,2000,青森県,NaN,"1,198,000","5,985","20,117",9,4,...,0.7,-11.8,25.7,10.7,15.0,3.2,2.5,0.7,3.1,1.39
3,2022000000,2022年,3000,岩手県,NaN,"1,173,000","5,788","19,342",15,6,...,1.0,-11.6,19.0,12.2,6.8,3.6,2.8,0.9,3.0,1.27
4,2022000000,2022年,4000,宮城県,NaN,"2,256,000","12,852","28,040",19,8,...,0.6,-6.7,20.7,9.8,10.8,2.9,2.3,0.5,3.7,1.35


In [ ]:
import pandas as pd

'''
データの前処理
'''
# 列名が扱いづらいから，変更する
rename_column_names = {
    '都道府県（特別区－指定都市再掲）': '都道府県',
    '時間軸(年次)': '年次',
    '人口【人】':'人口',
    '出生数【人】':'出生数',
    '死亡数【人】':'死亡数',
    '自然増減数【人】':'自然増減数'}
df = df.rename(columns=rename_column_names)

# 「全国，外国，不詳」などを排除し，リスト prefs に含まれる都道府県名の行のみを取り出す
jinko = df.copy()
jinko = jinko[df['都道府県'].isin(prefs)]

# 人数のデータにカンマが含まれるため，これらを削除し，整数型へ変換
# 年次については「2015年」を「2015」へ修正し，整数型へ変換
jinko['人口'] = jinko['人口'].str.replace(',', '').astype(int)
jinko['出生数'] = jinko['出生数'].str.replace(',', '').astype(int)
jinko['死亡数'] = jinko['死亡数'].str.replace(',', '').astype(int)
jinko['自然増減数'] = jinko['自然増減数'].str.replace(',', '').astype(int)
jinko['年次'] = jinko['年次'].str.replace('年', '').astype(int)

jinko= jinko[['年次', '都道府県', '人口', '出生数', '死亡数', '自然増減数' ]]

jinko # 確認のための出力

,年次,都道府県,人口,出生数,死亡数,自然増減数
1,2022,北海道,5098000,26407,74437,-48030
2,2022,青森県,1198000,5985,20117,-14132
3,2022,岩手県,1173000,5788,19342,-13554
4,2022,宮城県,2256000,12852,28040,-15188
5,2022,秋田県,926000,3992,17256,-13264
...,...,...,...,...,...,...
540,2015,熊本県,1777726,15577,20692,-5115
541,2015,大分県,1157581,9113,13958,-4845
542,2015,宮崎県,1100364,9226,13494,-4268
543,2015,鹿児島県,1642281,14125,21354,-7229


In [ ]:
# データフレーム jinko のデータ形式を確認
jinko.info()

<class 'pandas.core.frame.DataFrame'>
Index: 376 entries, 1 to 544
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   年次      376 non-null    int64 
 1   都道府県    376 non-null    object
 2   人口      376 non-null    int64 
 3   出生数     376 non-null    int64 
 4   死亡数     376 non-null    int64 
 5   自然増減数   376 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 20.6+ KB


---
### 山口県の過去8年間(2015〜2022年)の出生数，死亡数，自然増減数の推移

In [ ]:
# 山口県のデータのみを取り出す
yamaguchi = jinko[jinko['都道府県']=='山口県']
yamaguchi # 中身の確認

In [ ]:
# 出生数を棒グラフで表示


In [ ]:
# 死亡数を棒グラフで表示


In [ ]:
# 自然増減数を棒グラフで表示


---
### 各都道府県について出生率，死亡率，自然増減率を算出
- 出生率（出生数を人口で割った値）
- 死亡率（死亡数を人口で割った値）
- 自然増減率（出生数と死亡数の差を人口で割った値）

In [ ]:
# 出生率，死亡率，自然増減率を求める


---
### 2022年のデータに関して，出生率，死亡率，自然増減率それぞれでソートし，分析

In [ ]:
# 2022年のデータのみを取り出す
jinko_2022 = jinko[jinko['年次'] == 2022]
jinko_2022.tail()

,年次,都道府県,人口,出生数,死亡数,自然増減数,出生率,死亡率,自然増減率
43,2022,熊本県,1699000,11875,24427,-12552,0.006989,0.014377,-0.007388
44,2022,大分県,1092000,6798,16266,-9468,0.006225,0.014896,-0.008670
45,2022,宮崎県,1044000,7136,16111,-8975,0.006835,0.015432,-0.008597
46,2022,鹿児島県,1550000,10540,23925,-13385,0.006800,0.015435,-0.008635
47,2022,沖縄県,1446000,13594,15054,-1460,0.009401,0.010411,-0.001010


In [ ]:
# 出生率でソート


In [ ]:
# 死亡率でソート


In [ ]:
# 自然増減率でソート


---
### 2022年のデータから相関を見る

In [ ]:
# 横軸：出生率，縦軸：死亡率の散布図を描く


In [ ]:
# 横軸：人口，縦軸：出生率の散布図を描く


In [ ]:
# 横軸：人口，縦軸：死亡率の散布図を描く


---
### 各都道府県の自然増減率をヒートマップを用いて可視化

In [ ]:
# pivot を用いて jinko を並べ替える


In [ ]:
# ヒートマップの描画
